## This notebook was only created to test the ReadAndRunAllWorkFlows.py


<br>
<br>

In [1]:
import requests
import json
from collections import defaultdict
import os
import pandas as pd

In [2]:
def submit_to_ars(m,ars_url='https://ars.ci.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

In [3]:
def retrieve_ars_results(mid,ars_url='https://ars.ci.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results'
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error'
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error'
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results'
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return dictionary


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))

In [4]:
urls = ['/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/EGFR_advanced.json', '/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.1_RHOBTB2.json']

In [5]:
dict_workflows = {}
for i in urls:
    feature = (os.path.splitext(os.path.basename(i))[0])
    print(i, feature)
    with open(i,'r') as inf:
        query = json.load(inf)
        kcresult = submit_to_ars(query)
        #xx = retrieve_devars_results(kcresult)
        
        dict_workflows[feature] = kcresult

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/EGFR_advanced.json EGFR_advanced


ConnectionError: HTTPSConnectionPool(host='ars.ci.transltr.io', port=443): Max retries exceeded with url: /ars/api/submit (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x11d632d90>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [25]:
dict_workflows

{'EGFR_advanced': '3c2c11f1-3db7-4050-996d-b6dd19768296',
 'A.1_RHOBTB2': '325cfd91-ee6c-48eb-8a09-7226bbafb482'}

In [102]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    print(keys, val)
    
    result_status = retrieve_ars_results(val)
    
    workflow_result_messages[keys] = result_status

EGFR_advanced 3c2c11f1-3db7-4050-996d-b6dd19768296
Done
Done
kp-icees Done 0
Done
ara-explanatory Done 0
Error
kp-cohd Error 0
Done
kp-cam Done 0
Done
ara-improving ARS Error 0
Error
ara-aragorn Error 0
Done
kp-textmining Done 0
Done
ara-bte Done 0
Done
ara-unsecret Done 0
Done
kp-chp Done 0
Error
ara-arax Error 0
Done
kp-genetics Done 0
Done
kp-openpredict Done 0
Done
kp-molecular Done 0
A.1_RHOBTB2 325cfd91-ee6c-48eb-8a09-7226bbafb482
Done
Done
kp-cam Done 0
Done
ara-bte Done 8
Done
ara-aragorn Done 4
Done
kp-textmining Done 0
Done
kp-chp Done 0
Done
kp-icees Done 0
Done
ara-arax Done 9
Done
kp-openpredict Done 0
Done
ara-improving Done 4
Error
kp-cohd Error 0
Done
ara-unsecret Done 0
Done
kp-molecular Done 0
Done
kp-genetics Done 0
Done
ara-explanatory Done 0


In [103]:
workflow_result_messages

{'EGFR_advanced': {'pk_id': 'https://arax.ncats.io/?source=ARS&id=3c2c11f1-3db7-4050-996d-b6dd19768296',
  'kp-icees': 'No Results',
  'ara-explanatory': 'No Results',
  'kp-cohd': 'Error',
  'kp-cam': 'No Results',
  'ara-improving': 'ARS Error',
  'ara-aragorn': 'Error',
  'kp-textmining': 'No Results',
  'ara-bte': 'No Results',
  'ara-unsecret': 'No Results',
  'kp-chp': 'No Results',
  'ara-arax': 'Error',
  'kp-genetics': 'No Results',
  'kp-openpredict': 'No Results',
  'kp-molecular': 'No Results'},
 'A.1_RHOBTB2': {'pk_id': 'https://arax.ncats.io/?source=ARS&id=325cfd91-ee6c-48eb-8a09-7226bbafb482',
  'kp-cam': 'No Results',
  'ara-bte': 'Results',
  'ara-aragorn': 'Results',
  'kp-textmining': 'No Results',
  'kp-chp': 'No Results',
  'kp-icees': 'No Results',
  'ara-arax': 'Results',
  'kp-openpredict': 'No Results',
  'ara-improving': 'Results',
  'kp-cohd': 'Error',
  'ara-unsecret': 'No Results',
  'kp-molecular': 'No Results',
  'kp-genetics': 'No Results',
  'ara-explan

In [159]:
col = []
final_dict = defaultdict(list)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k].items():
        if key.startswith('kp-'):
            key_mod = key.replace('kp-','')
        else:
            key_mod = key
        
        final_dict[key_mod].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

A.1_RHOBTB2
EGFR_advanced


In [160]:
df

,A.1_RHOBTB2,EGFR_advanced
pk_id,https://arax.ncats.io/?source=ARS&id=325cfd91-...,https://arax.ncats.io/?source=ARS&id=3c2c11f1-...
cam,No Results,No Results
ara-bte,Results,No Results
ara-aragorn,Results,Error
textmining,No Results,No Results
chp,No Results,No Results
icees,No Results,No Results
ara-arax,Results,Error
openpredict,No Results,No Results
ara-improving,Results,ARS Error


In [54]:
df.loc['pk_id']

A.1_RHOBTB2      https://arax.ncats.io/?source=ARS&id=325cfd91-...
EGFR_advanced    https://arax.ncats.io/?source=ARS&id=3c2c11f1-...
Name: pk_id, dtype: object

In [58]:
df.to_excel('test_ara_worklow.xlsx')

In [ ]:
pwd

In [ ]:
## test for individual run

In [11]:
with open('/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowC/C.1_ChemSubstances_related_to_Disease.json','r') as inf:
    query1 = json.load(inf)

In [12]:
printjson(query1)

{
    "message": {
        "query_graph": {
            "edges": {
                "e00": {
                    "subject": "n00",
                    "object": "n01",
                    "predicates": [
                        "biolink:related_to"
                    ]
                }
            },
            "nodes": {
                "n00": {
                    "categories": [
                        "biolink:ChemicalSubstance"
                    ]
                },
                "n01": {
                    "categories": [
                        "biolink:Disease"
                    ],
                    "ids": [
                        "MONDO:0005301"
                    ]
                }
            }
        }
    }
}


In [13]:
kcresult = submit_to_ars(query1)

https://arax.ncats.io/?source=ARS&id=e9ff8f3d-9a4e-4bc1-9dd6-53f4f4c9485f


In [14]:
result_status = retrieve_ars_results(kcresult)

Running
Running
ara-arax Running 0
Done
ara-improving Done 176
Running
kp-textmining Running 0
Done
kp-icees Done 0
Running
ara-unsecret Running 0
Done
kp-cohd Done 500
Running
ara-aragorn Running 0
Running
kp-openpredict Running 0
Done
kp-genetics Done 0
Running
ara-bte Running 0
Running
kp-cam Running 0
Done
ara-explanatory Done 15
Done
kp-molecular Done 15
Done
kp-chp Done 0


In [21]:
result_status 

{'pk_id': 'https://arax.ncats.io/?source=ARS&id=737fe659-c952-422a-91ef-4f088ca7f41d',
 'kp-openpredict': 'No Results',
 'ara-bte': 'Results',
 'ara-arax': 'Results',
 'kp-cam': 'No Results',
 'ara-improving': 'Results',
 'ara-aragorn': 'Results',
 'ara-explanatory': 'No Results',
 'kp-textmining': 'No Results',
 'kp-icees': 'No Results',
 'kp-genetics': 'No Results',
 'kp-chp': 'No Results',
 'ara-unsecret': 'No Results',
 'kp-cohd': 'Error',
 'kp-molecular': 'No Results'}

In [9]:
pwd

'/Users/priyash/Documents/GitHub/minihackathons/Notebooks'

In [112]:
df

,A.1_RHOBTB2,EGFR_advanced
pk_id,https://arax.ncats.io/?source=ARS&id=325cfd91-...,https://arax.ncats.io/?source=ARS&id=3c2c11f1-...
kp-cam,No Results,No Results
ara-bte,Results,No Results
ara-aragorn,Results,Error
kp-textmining,No Results,No Results
kp-chp,No Results,No Results
kp-icees,No Results,No Results
ara-arax,Results,Error
kp-openpredict,No Results,No Results
ara-improving,Results,ARS Error


In [143]:
def make_hyperlink(value):
    #url = "https://custom.url/{}"
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

In [144]:
make_hyperlink('https://arax.ncats.io/?source=ARS&id=325cfd91-ee6c-48eb-8a09-7226bbafb482')

'=HYPERLINK("https://arax.ncats.io/?source=ARS&id=325cfd91-ee6c-48eb-8a09-7226bbafb482", "https://arax.ncats.io/?source=ARS&id=325cfd91-ee6c-48eb-8a09-7226bbafb482")'

In [148]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [149]:
df

,A.1_RHOBTB2,EGFR_advanced
pk_id,"=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&..."
kp-cam,No Results,No Results
ara-bte,Results,No Results
ara-aragorn,Results,Error
kp-textmining,No Results,No Results
kp-chp,No Results,No Results
kp-icees,No Results,No Results
ara-arax,Results,Error
kp-openpredict,No Results,No Results
ara-improving,Results,ARS Error


In [150]:
df.to_excel('test_ara_worklow.xlsx')

In [151]:
df

,A.1_RHOBTB2,EGFR_advanced
pk_id,"=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&..."
kp-cam,No Results,No Results
ara-bte,Results,No Results
ara-aragorn,Results,Error
kp-textmining,No Results,No Results
kp-chp,No Results,No Results
kp-icees,No Results,No Results
ara-arax,Results,Error
kp-openpredict,No Results,No Results
ara-improving,Results,ARS Error


In [154]:
x = 'kp-cam'
if x.startswith('kp-'):
    print(x.replace('kp-',''))
else:
    print('nothing')
    

cam
